In [129]:
import sys
import numpy as np
from mpi4py import MPI
import pandas as pd
import time
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as ss
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from itertools import product
#import matplotlib.pyplot as plt
#from imblearn.over_sampling import SMOTENC

## Method 1

In [139]:
%%writefile rf_tuning.py

import numpy as np
from numpy import pi, exp, sqrt
from mpi4py import MPI
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
#from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

X = pd.read_csv('input.csv')
y = X['target']
X = X.drop(columns = ['target'])
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

start = time.time()

n_estimators = [50,100]
max_features = ['sqrt','log2']
max_depth = [5,10]
min_samples_split = [10,20]
param_grid_prod = list(product(n_estimators,max_features,max_depth,min_samples_split))
num_params = len(param_grid_prod)

def rf_model (X_train,y_train,X_test,y_test,param):
    """
    Args:
    parm = [max_depth, n_estimators, 'min_samples_split', 'max_features']
    
    """
    rf = RandomForestClassifier(n_estimators = param[0],max_features = param[1],max_depth = param[2], min_samples_split = param[3])
    #rf.grid = GridSearchCV(rf, param_grid,cv = 5,scoring = 'roc_auc')
    rf.fit(X_train, y_train)
    y_test_prob = rf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_test_prob)
    #print(param, auc)
    return auc

#n = np.array(num_params)
auc = np.zeros(1)
auc_list = np.zeros(num_params)
param_idx = np.zeros(num_params)
auc_rank = np.zeros(size)
param_list = []

# Compute partition
for i in range(num_params):
    if rank == i%size:
        auc = rf_model (X_train, y_train, X_test, y_test,param_grid_prod[i])
        #param_list.append(param_grid_prod[i])
        auc_list[i] = auc
        param_idx[i] = i

for i in range(size):
    if rank == i:
        print('Rank: ', rank)
        print('max_auc of current rank: ', np.max(auc_list))
        #auc_rank[i] = np.max(auc_list)
        #print('whole auc_rank: ',auc_rank)
        max_idx = np.argmax(auc_list)
        print('best parameter combincation: ',param_grid_prod[max_idx])
        #param_list.append(param_grid_prod[max_idx])
        
end = time.time()
print('Time elapsed: ', end - start)
        
"""        
print('auc:',auc_rank)
print('param:',param_list)

if rank == 0:
    print('Rank: ',rank)
    print(max(auc_list))
    print(np.argmax(auc_list))

if rank == 1:
    print('Rank: ', rank)
    print(max(auc_list))
    print(np.argmax(auc_list))


max_idx = np.argmax(auc_list)
print('max_idx:',max_idx)
print('param:',param_grid_prod[max_idx])
print('best_auc:',auc_list[max_idx])
print('param_list:', len(param_list))
"""

Overwriting rf_tuning.py


In [138]:
!mpirun -n 4 python3 rf_tuning.py

## Method 2

In [133]:
%%writefile rf_tuning.py

import numpy as np
from numpy import pi, exp, sqrt
from mpi4py import MPI
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
#from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

X = pd.read_csv('input.csv')
y = X['target']
X = X.drop(columns = ['target'])
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

n_estimators = [50,100]
max_features = ['sqrt','log2']
max_depth = [5,10]
min_samples_split = [10,20]
param_grid_prod = list(product(n_estimators,max_features,max_depth,min_samples_split))
num_params = len(param_grid_prod)
rank_size = num_params // size

def rf_model (X_train,y_train,X_test,y_test,param):
    """
    Args:
    parm = [max_depth, n_estimators, 'min_samples_split', 'max_features']
    
    """
    rf = RandomForestClassifier(n_estimators = param[0],max_features = param[1],max_depth = param[2], min_samples_split = param[3])
    #rf.grid = GridSearchCV(rf, param_grid,cv = 5,scoring = 'roc_auc')
    rf.fit(X_train, y_train)
    y_test_prob = rf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_test_prob)
    print(param, auc)
    return auc

#n = np.array(num_params)
auc = np.zeros(1)
#auc_list = np.zeros(num_params)
#param_idx = np.zeros(num_params)
auc_rank = np.zeros(size)
#param_rank = []
#param_list = []
best_auc = np.zeros(1)
param_rank = np.empty(4, dtype = 'object')
param_result = np.empty(4, dtype = 'object')


if rank == (size-1):
    sub_param_grid = param_grid_prod[rank*rank_size:num_params]
    auc_result = []
    #auc_result = np.zeros(N - rank*rank_size)
    for param in sub_param_grid:
        auc = rf_model(X_train, y_train, X_test, y_test,param)
        auc_result.append(auc)
    best_auc = np.max(auc_result)
    idx = np.argmax(best_auc)
    best_param = sub_param_grid[idx]
    #comm.Gather(best_auc, auc_rank, root=0)
    print('Rank: ',rank, 'Best AUC: ', best_auc, 'Best Parameter: ',best_param)

else:
    sub_param_grid = param_grid_prod[rank*rank_size:(rank+1)*rank_size]
    auc_result = []
    param_list = []
    for i in range(len(sub_param_grid)):
        auc = rf_model(X_train, y_train, X_test, y_test,sub_param_grid[i])
        auc_result.append(auc)
    
    best_auc = np.max(auc_result)
    idx = np.argmax(best_auc)
    best_param = sub_param_grid[idx]
    print('Rank: ',rank, 'Best AUC: ', best_auc, 'Best Parameter: ',best_param)


#param_rank.append(best_param)
        
#comm.Gather(best_auc, auc_rank, root=0)
#comm.Gather(param_result,param_rank, root=0)


# Only print the result in process 0
#if rank == 0:
#    print('final_result: ',auc_rank)
#    print('best_parameter: ',param_result)
    #print(best_param)

        

Overwriting rf_tuning.py


In [135]:
"""
%%writefile rf_tuning.py

import numpy as np
from numpy import pi, exp, sqrt
from mpi4py import MPI
import time
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from itertools import product
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, roc_auc_score
#from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split


comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

X = pd.read_csv('input.csv')
y = X['target']
X = X.drop(columns = ['target'])
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

n_estimators = [50,100]
max_features = ['sqrt','log2']
max_depth = [5,10]
min_samples_split = [10,20]
param_grid_prod = list(product(n_estimators,max_features,max_depth,min_samples_split))
num_params = len(param_grid_prod)
rank_size = num_params // size

def rf_model (X_train,y_train,X_test,y_test,param):
    """
    Args:
    parm = [max_depth, n_estimators, 'min_samples_split', 'max_features']
    
    """
    rf = RandomForestClassifier(n_estimators = param[0],max_features = param[1],max_depth = param[2], min_samples_split = param[3])
    #rf.grid = GridSearchCV(rf, param_grid,cv = 5,scoring = 'roc_auc')
    rf.fit(X_train, y_train)
    y_test_prob = rf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_test_prob)
    print(param, auc)
    return auc

#n = np.array(num_params)
auc = np.zeros(1)
#auc_list = np.zeros(num_params)
#param_idx = np.zeros(num_params)
auc_rank = np.zeros(size)
#param_rank = []
#param_list = []
best_auc = np.zeros(1)
#param_rank = np.empty(4, dtype = 'object')
#param_result = np.empty(4, dtype = 'object')
#best_param = np.empty(4, dtype = 'object')


if rank == (size-1):
    sub_param_grid = param_grid_prod[rank*rank_size:num_params]
    auc_result = []
    #auc_result = np.zeros(N - rank*rank_size)
    for param in sub_param_grid:
        auc = rf_model(X_train, y_train, X_test, y_test,param)
        auc_result.append(auc)
    best_auc = np.max(auc_result)
    #comm.Gather(best_auc, auc_rank, root=0)

else:

sub_param_grid = param_grid_prod[rank*rank_size:(rank+1)*rank_size]
auc_result = []
param_list = []
for i in range(len(sub_param_grid)):
    auc = rf_model(X_train, y_train, X_test, y_test,sub_param_grid[i])
    auc_result.append(auc)
    
best_auc = np.max(auc_result)
idx = np.argmax(best_auc)
best_param = sub_param_grid[idx]
param_result = (rank, best_param)
#print('Rank: ',rank, 'Best AUC: ', best_auc, 'Best Parameter: ',best_param)


#param_rank.append(best_param)
        
#comm.Gather(best_auc, auc_rank, root=0)
comm.gather(param_result, root=0)


# Only print the result in process 0
if rank == 0:
    for i in range(size):
        print(param_result[i])
#    print('final_result: ',auc_rank)
    #print('best_parameter: ',param_result)
    #print(best_param)
"""
        

Overwriting rf_tuning.py
